In [1]:
import re

import numpy as np
import pandas as pd

# Lectura

In [2]:
data_trial = pd.read_csv("../data/tsd_trial.csv")
print(data_trial.shape)
data_trial.head()

(690, 2)


,spans,text
0,"[15, 16, 17, 18, 19, 27, 28, 29, 30, 31]",Because he's a moron and a bigot. It's not any...
1,"[29, 30, 31, 32, 33, 34]",How about we stop protecting idiots and let na...
2,"[166, 167, 168, 169, 170, 171]","If people were smart, they would Boycott th..."
3,"[87, 88, 89, 90, 91, 92]",Trump Claimed that Russia will never invade th...
4,[],As long as your willing to pay a lot more for ...


In [3]:
data_train = pd.read_csv("../data/tsd_train.csv")
print(data_train.shape)
data_train

(7939, 2)


,spans,text
0,"[8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19,...",Another violent and aggressive immigrant killi...
1,"[33, 34, 35, 36, 37, 38, 39]","I am 56 years old, I am not your fucking junio..."
2,"[0, 1, 2, 3]","Damn, a whole family. Sad indeed."
3,"[7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17]",What a knucklehead. How can anyone not know th...
4,"[32, 33, 34, 35, 36, 37, 38]","""who do you think should do the killing?""\n\nA..."
...,...,...
7934,"[8, 9, 10, 11]",Another fool pipes in.
7935,"[48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 5...",So if a restaurant owner puts up a sign saying...
7936,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",Any faith that can't stand up to logic and rea...
7937,"[5, 6, 7, 8, 9, 10, 11]",This idiotic. Use the surplus to pay down the ...


In [4]:
data = pd.concat([data_trial, data_train])
data.shape

(8629, 2)

Vemos que había solamente cinco textos repetidos. 

Se usará `tsd_train.csv` para entrenar y `tsd_trial` para validación

-------------

# Alfabeto

Vendría bien conocer que símbolos hay en los textos

In [5]:
conjuntos = data_train.text.agg(set)
alfabeto = set()

In [6]:
for index,item in conjuntos.iteritems():
    alfabeto |= item

alfabeto

{'\t',
 '\n',
 ' ',
 '!',
 '"',
 '#',
 '$',
 '%',
 '&',
 "'",
 '(',
 ')',
 '*',
 '+',
 ',',
 '-',
 '.',
 '/',
 '0',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 ':',
 ';',
 '<',
 '=',
 '>',
 '?',
 '@',
 'A',
 'B',
 'C',
 'D',
 'E',
 'F',
 'G',
 'H',
 'I',
 'J',
 'K',
 'L',
 'M',
 'N',
 'O',
 'P',
 'Q',
 'R',
 'S',
 'T',
 'U',
 'V',
 'W',
 'X',
 'Y',
 'Z',
 '[',
 '\\',
 ']',
 '^',
 '_',
 '`',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z',
 '{',
 '}',
 '~',
 '\x7f',
 '\xa0',
 '«',
 '¬',
 '·',
 '»',
 'é',
 'ê',
 'ï',
 'ó',
 'ü',
 'ʻ',
 'ˈ',
 '͞',
 '\u2004',
 '\u200b',
 '–',
 '—',
 '―',
 '‘',
 '’',
 '“',
 '”',
 '•',
 '…',
 '™',
 '▀',
 '☠',
 '☭',
 '☹',
 '⚽',
 '⚾',
 '✭',
 '️',
 '🆘',
 '👎',
 '💀',
 '💥',
 '💨',
 '🔥',
 '😁',
 '😂',
 '😅',
 '😆',
 '😈',
 '😉',
 '😊',
 '😜',
 '😞',
 '😡',
 '😬',
 '😵',
 '🙄',
 '🤥'}

In [7]:
len(alfabeto)

150

Aparte del abecedario, tenemos algunos símbolos que se usan con relativa frecuencia en internet.

Sin embargo, también notamos que hay caracteres raros como las comillas dobles, guiones con sus variantes, etc. También hay que notar que hay emojis 

# Vocabulario

Para obtener el vocabulario, es necesario realizar un preprocesamiento a los textos, esto incluye:
* sustituir minúsculas
* quitar emojis
* ¿quitar caracteres raros?

In [8]:
def limpiar(texto):
    texto = texto.lower() #minúsculas
    texto = re.sub("[\W_]", " ", texto) #carácteres raros
    texto = re.sub("\s+", " ", texto) # colapsando espaciados
    return texto

In [9]:
s = data_train.text[0]
s

'Another violent and aggressive immigrant killing a innocent and intelligent US Citizen.... Sarcasm'

In [10]:
limpiar(s)

'another violent and aggressive immigrant killing a innocent and intelligent us citizen sarcasm'

In [11]:
texto_preprocesado = data_train.text.apply(limpiar)

texto_tokenizado = texto_preprocesado.apply(str.split)

vocabulario = set()
for index, lista in texto_tokenizado.iteritems():
    vocabulario |= set(lista)

len(vocabulario)

18992

# Preprocesamiento

Vamos a obtener el texto que se ha considerado como tóxico

In [12]:
def procesar_span(span):
    span = re.sub("\s","",span[1:-1])
    if span:
        lista = span.split(",")
        lista = [int(index) for index in lista]
    else:
        lista = []
    return lista

In [13]:
data_train["spans"] = data_train["spans"].apply(procesar_span)

In [14]:
def obtener_texto_toxico(row):
    toxico = []
    if row.spans: # Hay texto marcado como toxico
        diffs = np.diff(row.spans, prepend=row.spans[0]-1) #prepend para que el primer diff sea uno
        toxico = []
        indices_inicios = np.argwhere(diffs != 1).ravel() # indices de los inicios de las sigs palabras

        i = row.spans[0]
        for indice_next in indices_inicios:
            j = row.spans[indice_next-1] # fin de la anterior
            parte_toxica = row.text[i:j+1]
            toxico.append(parte_toxica)
            i = row.spans[indice_next]
            
        # Última parte
        j = row.spans[-1]
        parte_toxica = row.text[i:j+1]
        toxico.append(parte_toxica)

    return toxico

## Quitando espacios de los tramos tóxicos

In [15]:
def quitar_espacios_de_spans(row):
    toxico = obtener_texto_toxico(row)
    toxic_span = "".join(toxico) # juntar texto tóxico
    toxic_match = set(re.findall(r"\W", toxic_span)) # encontrar cosas que no son letras, números o guión bajo
    # Inicialización
    buenos = row.spans
    indices = []
    
    for extraño in toxic_match: # Si no hay nada en toxic_match, no se itera
        extraño = re.escape(extraño) # escapando posibles caracteres con interpretación RegEx
        if extraño == "\ ":
            indice = re.finditer(extraño, toxic_span) 
            indices = [i.start() for i in indice] # Índices de espacios en el texto
    if indices:
        por_quitar = [row.spans[i] for i in indices] # spans tóxicos que son espacios
        buenos = [i for i in row.spans if i not in por_quitar] # filtrando
            
    return buenos

In [16]:
row = data_train.iloc[0]
print(row.spans)
row.text

[8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39]


'Another violent and aggressive immigrant killing a innocent and intelligent US Citizen.... Sarcasm'

In [17]:
print(quitar_espacios_de_spans(row))

[8, 9, 10, 11, 12, 13, 14, 16, 17, 18, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 31, 32, 33, 34, 35, 36, 37, 38, 39]


# Quitando puntuación

No es tan trivial el solamente eliminar la puntuación y cosas raras, tal como se hizo al inicio al momento de querer el vocabulario. Se debe registrar donde se están eliminando cosas para luego ver si eso era parte de un toxic span

In [18]:
def eliminar_caracteres(row):
    #toxico = obtener_texto_toxico(row)
    #toxic_span = "".join(toxico) # juntar texto tóxico
    strange_chars = set(re.findall(r"[^\w\-*&#'¿?¡!%`<>]", row.text)) # encontrar cosas que no son letras, números o guión bajo
    # Inicialización
    offsets_filtrado = row.spans
    texto_filtrado = row.text
    indices = []
    
    for extraño in strange_chars: # Si no hay nada en toxic_match, no se itera
        extraño = re.escape(extraño) # escapando posibles caracteres con interpretación RegEx            
        extraño_matchs = re.finditer(extraño, row.text)  # busca los caracteres raros
        indices += [i.start() for i in extraño_matchs] # Índices de caracteres en el texto
        
    if indices:
        indices = set(indices)
        spans_malos = indices & set(row.spans)
        offsets_filtrado = [i for i in row.spans if i not in spans_malos] # filtrando
        texto_filtrado = [c for i,c in enumerate(row.text) if i not in indices or c == " "]
        texto_filtrado = "".join(texto_filtrado)
            
    return offsets_filtrado, texto_filtrado

In [19]:
row = data_train.loc[7938]
print(row.spans)
print(row.text)

[106, 107, 108, 109, 110, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182]
Who is this "we" of which you speak? Are you really a woman, Lars? If not then concern yourself with your penis and other genital and reproductive parts and stay out of women's bodies.


In [20]:
print(eliminar_caracteres(row))

([106, 107, 108, 109, 110, 169, 170, 171, 172, 173, 174, 175, 177, 178, 179, 180, 181, 182], "Who is this we of which you speak? Are you really a woman Lars? If not then concern yourself with your penis and other genital and reproductive parts and stay out of women's bodies")


## Corrigiendo errores de los pendejos que subrayaron

In [21]:
def separar_spans_toxicos(row):
    lists = []
    start = row.spans[0]
    end = row.spans[0]
    for i in range(1, len(row.spans)):
        #print(row.spans[i])
        end = row.spans[i]
        previous = row.spans[i-1] 
        if (end - previous) != 1: # Terminó un span, inició el otro
            lists.append(list(range(start,previous+1)))
            start = end
    
    # Agregando último span
    lists.append(list( range(start, end+1) ))
            
    return lists

In [22]:
def corregir_mamadas(row, umbral):
    lista_palabras = row.text.split()
    starts, ends = [], []
    start, end = 0, -2
    for idx, palabra in enumerate(lista_palabras):
        starts.append(start) # start actual
        start += len(palabra) + 1 # start para la siguiente
        end   += len(palabra) + 1 # end actual
        ends.append(end)
    
    toxicos = []
    if row.spans:
        indices_separados = separar_spans_toxicos(row)
        for separado in indices_separados: # iterando partes tóxicas 
            indices = separado
            for s, e in zip(starts, ends): # iterando "texto" por medio de los índices de palabras
                if separado[0] in range(s,e+1): # span está en una palabra
                    if separado != list(range(s, e+1)): # los spans no abarca toda la palabra
                        #print(row.text[s:e+1])
                        if len(separado)/(len(list(range(s, e+1)))) <= umbral:
                            #print("No será marcado\n")
                            indices = [] # mejor ni lo marques
                        else:
                            #print("Será marcado en su totalidad")
                            indices = list(range(s, e+1)) # ya márcate todo mejor
                    else:
                        break # la palabra está bien marcada, pasa a la siguiente parte tóxica
            toxicos.extend(indices)
    return toxicos

## Probando con el ejemplo de Tradeu:

In [23]:
row = data_train.iloc[218]
print(row.spans)
row.text

[6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149]


"Trudeau is full of crap. The truth is he only lost interest in electoral reform when it was pointed out to His Royal Anus that it wouldn't give the Liberals the unfair advantage he sought."

In [24]:
row.spans = quitar_espacios_de_spans(row)
print(row.spans)

[6, 8, 9, 11, 12, 13, 14, 16, 17, 19, 20, 21, 22, 23, 25, 26, 27, 29, 30, 31, 32, 33, 35, 36, 38, 39, 41, 42, 43, 44, 46, 47, 48, 49, 51, 52, 53, 54, 55, 56, 57, 58, 60, 61, 63, 64, 65, 66, 67, 68, 69, 70, 71, 73, 74, 75, 76, 77, 78, 80, 81, 82, 83, 85, 86, 88, 89, 90, 92, 93, 94, 95, 96, 97, 98, 100, 101, 102, 104, 105, 107, 108, 109, 111, 112, 113, 114, 115, 117, 118, 119, 120, 122, 123, 124, 125, 127, 128, 130, 131, 132, 133, 134, 135, 136, 137, 139, 140, 141, 142, 144, 145, 146, 148, 149]


In [25]:
# Antes: Sobran el índice 6, el 148 y el 149
print(corregir_mamadas(row, 0.75))

[8, 9, 11, 12, 13, 14, 16, 17, 19, 20, 21, 22, 23, 25, 26, 27, 29, 30, 31, 32, 33, 35, 36, 38, 39, 41, 42, 43, 44, 46, 47, 48, 49, 51, 52, 53, 54, 55, 56, 57, 58, 60, 61, 63, 64, 65, 66, 67, 68, 69, 70, 71, 73, 74, 75, 76, 77, 78, 80, 81, 82, 83, 85, 86, 88, 89, 90, 92, 93, 94, 95, 96, 97, 98, 100, 101, 102, 104, 105, 107, 108, 109, 111, 112, 113, 114, 115, 117, 118, 119, 120, 122, 123, 124, 125, 127, 128, 130, 131, 132, 133, 134, 135, 136, 137, 139, 140, 141, 142, 144, 145, 146]


## Ahora sí en general:

In [38]:
def preprocesar(row):
    offsets, texto = eliminar_caracteres(row)
    new_row = pd.Series([offsets, texto], index=row.index)
    new_row.spans = corregir_mamadas(new_row, 0.75)
    new_row.text = new_row.text.lower()
    return new_row

In [31]:
def preprocesar(row):
    offsets = quitar_espacios_de_spans(row)
    new_row = pd.Series([offsets, row.text.lower()], index=row.index)
    new_row.spans = corregir_mamadas(new_row, 0.75)
    return new_row

In [39]:
train_cleaned = data_train.apply(preprocesar, axis=1)

In [40]:
train_cleaned

,spans,text
0,"[8, 9, 10, 11, 12, 13, 14, 16, 17, 18, 20, 21,...",another violent and aggressive immigrant killi...
1,"[32, 33, 34, 35, 36, 37, 38]",i am 56 years old i am not your fucking junior...
2,"[0, 1, 2, 3]",damn a whole family sad indeed
3,"[7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17]",what a knucklehead how can anyone not know thi...
4,[],who do you think should do the killing?anyone ...
...,...,...
7934,"[8, 9, 10, 11]",another fool pipes in
7935,"[47, 48, 50, 51, 52, 53, 54, 55, 57, 58, 59, 6...",so if a restaurant owner puts up a sign saying...
7936,"[0, 1, 2, 4, 5, 6, 7, 8, 10, 11, 12, 13, 15, 1...",any faith that can't stand up to logic and rea...
7937,"[5, 6, 7, 8, 9, 10, 11]",this idiotic use the surplus to pay down the p...


No se ven diferencias porque habían pocas de estas situaciones, o al menos sólo observamos una, la de Tradeu.

In [34]:
train_cleaned.to_csv("../data/train_cleaned_exp.csv", index = False)

In [35]:
data_trial.spans = data_trial.spans.apply(procesar_span)

In [36]:
trial_cleaned = data_trial.apply(preprocesar, axis=1)

In [37]:
data_trial.to_csv("../data/trial_cleaned.csv", index = False)

-----------

# Etiquetado POS

In [29]:
from nltk.tag import pos_tag
from nltk.tokenize import word_tokenize
import nltk
#nltk.download('punkt')
#nltk.download('averaged_perceptron_tagger')
text = nltk.word_tokenize("Do you own stock in one of Trump's Russian enterprises or do you just not care that a presidential election was hijacked? ")
nltk.pos_tag(text)

[('Do', 'VB'),
 ('you', 'PRP'),
 ('own', 'VB'),
 ('stock', 'NN'),
 ('in', 'IN'),
 ('one', 'CD'),
 ('of', 'IN'),
 ('Trump', 'NNP'),
 ("'s", 'POS'),
 ('Russian', 'JJ'),
 ('enterprises', 'NNS'),
 ('or', 'CC'),
 ('do', 'VB'),
 ('you', 'PRP'),
 ('just', 'RB'),
 ('not', 'RB'),
 ('care', 'VB'),
 ('that', 'IN'),
 ('a', 'DT'),
 ('presidential', 'JJ'),
 ('election', 'NN'),
 ('was', 'VBD'),
 ('hijacked', 'VBN'),
 ('?', '.')]

In [31]:
#nltk.pos_tag?

In [32]:
#nltk.pos_tag_sents?